# Going Modular

Turn notebook into a series of different python scripts. For example:

* data_setup.py - a file to prepare and download data if needed.
* engine.py - a file containing various training functions.
* model_builder.py or model.py - a file to create a PyTorch model.
* train.py - a file to leverage all other files and train a target PyTorch model.
* utils.py - a file dedicated to helpful utility functions.

Why?

* More reproducible and easier to run in larg scale projects.

Example workflow:
* Start machine learning projects in Jupyter/Google Colab notebooks for quick experiment and visualization.
* When got something working, move the most useful pieces of code to python scripts.

Note:
* Docstrings: To describe each of the functions/classes that will be put into scripts.
* Imports at the top of scripts.

# Get Data

In [2]:
import os
import torch
import requests, zipfile

from pathlib import Path
from torch import nn
from torchvision import transforms

In [3]:
# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza-steak-sushi"

# If the image folder doens't exist, download
if image_path.is_dir():
    print(f"{image_path} directory exists")
else:
    print(f"Didn't find {image_path}, creating one ...")
    image_path.mkdir(parents =  True, exist_ok = True)

# Download
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data ...")
    f.write(request.content)

# Unzip
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data ...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Didn't find data/pizza-steak-sushi, creating one ...
Unzipping pizza, steak, sushi data ...


In [20]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders
for image classification data
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
):
    ''' Creates training and testing DataLoaders

    Input: Training directory and testing directory path
    -> Pytorch Datasets
    -> Output: Pytorch DataLoaders

    Args:
        train_dir: Path to training directory
        test_dir: Path to testing directory
        transform: Transforms to perform on training and testing data
        batch_size: Number of samples per batch in each of DataLoaders
        num_workers: Number of workers per DataLoader

    Returns:
        (train_dataloader, test_dataloader, class_names)
    '''

    # 1. Create datasets from original directory
    train_data = datasets.ImageFolder(train_dir, transform = transform)
    test_data = datasets.ImageFolder(test_dir, transform = transform)

    # 2. Turn data into dataloaders
    train_dataloader = DataLoader(
        train_data,
        batch_size = batch_size,
        shuffle = True,
        num_workers = num_workers,
        pin_memory = True
    )

    test_dataloader = DataLoader(
        test_data,
        batch_size = batch_size,
        shuffle = False,
        num_workers = num_workers,
        pin_memory = True
    )

    return train_dataloader, test_dataloader, train_data.classes

Overwriting going_modular/data_setup.py


# Building Model

In [10]:
%%writefile going_modular/model_builder.py
'''
Contains PyTorch code to instantiate a TinyVGG Model.
'''
import torch
from torch import nn

class TinyVGG(nn.Module):
    ''' Creates the TinyVGG architecture.

    Args:
        input_shape: Number of input channels
        hidden_units: Number of hidden units between layers
        output_shape: Number of output units
    '''
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
              in_channels = input_shape,
              out_channels = hidden_units,
              kernel_size = 3,
              stride = 1,
              padding = 0
            ),
            nn.ReLU(),
            nn.Conv2d(
              in_channels = hidden_units,
              out_channels = hidden_units,
              kernel_size = 3,
              stride = 1,
              padding = 0
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size = 2,
                stride = 2
            )
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
              in_channels = hidden_units,
              out_channels = hidden_units,
              kernel_size = 3,
              padding = 0
            ),
            nn.ReLU(),
            nn.Conv2d(
              in_channels = hidden_units,
              out_channels = hidden_units,
              kernel_size = 3,
              padding = 0
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size = 2
            )
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(
                in_features = hidden_units * 13 * 13,
                out_features = output_shape
            )
        )

    def forward(self, x: torch.Tensor):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Writing going_modular/model_builder.py


# Training and Testing

In [26]:
%%writefile going_modular/engine.py
'''
Contains functions for training and testing a pytorch model
'''
import torch

from torch import nn
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device
) -> Tuple[float, float]:
    ''' Trains a PyTorch model for a single epoch.

    Args:
        model: A PyTorch model to train
        dataloader: A DataLoader instance for the model to be trained on
        loss_fn: A PyTorch loss function to minimize
        optimizer: A PyTorch optimizer to help minimize the loss function
        device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
        (train_loss, train_accuracy).
    '''
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 1. Forward-pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Calculate gradients for back prop
        loss.backward()

        # 5. Updates parameters
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim = 1), dim = 1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss /= len(dataloader)
    train_acc /= len(dataloader)

    return train_loss, train_acc

def test_step(
    model: nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    device: torch.device
) -> Tuple[float, float]:
    ''' Tests a PyTorch model for a single epoch.

    Args:
        model: A PyTorch model to train
        dataloader: A DataLoader instance for the model to be trained on
        loss_fn: A PyTorch loss function to minimize
        device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
        (test_loss, test_accuracy).
    '''
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # 1. Forward-pass
            test_pred_logits = model(X)

            # 2. Calculate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # 3. Calculate accuracy
            test_pred_labels = torch.argmax(torch.softmax(test_pred_logits, dim = 1), dim = 1)
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

    # Adjust metrics to get average loss and accuracy per batch
    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

    return test_loss, test_acc

def train(
    model: nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device
) -> Dict[str, List]:
    ''' Trains a PyTorch model.

    Args:
        model: A PyTorch model to train
        train_dataloader: A DataLoader instance for the model to be trained on
        test_dataloader: A DataLoader instance for the model to be tested on
        loss_fn: A PyTorch loss function to minimize
        optimizer: A PyTorch optimizer to help minimize the loss function
        epochs: Number of epochs to train for
        device: A target device to compute on (e.g. "cuda" or "cpu")

    Returns: History of training and testing loss and accuracy per epoch
        {train_loss: [...],
         train_acc: [...],
         test_loss: [...],
         test_acc: [...]}
    '''
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model = model,
            dataloader = train_dataloader,
            loss_fn = loss_fn,
            optimizer = optimizer,
            device = device
        )
        test_loss, test_acc = test_step(
            model = model,
            dataloader = test_dataloader,
            loss_fn = loss_fn,
            device = device
        )

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

Overwriting going_modular/engine.py


# Save and Load Model

In [13]:
%%writefile going_modular/utils.py
'''
Contains utility functions for PyTorch model training and saving.
'''
import torch

from torch import nn
from pathlib import Path

def save_model(
    model: nn.Module,
    target_dir: str,
    model_name: str
):
    ''' Saves a PyTorch model to a target directory

    Args:
        model: A target PyTorch model to save
        target_dir: A directory for saving the model to
        model_name: A filename for the saved model
    '''
    # 1. Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents = True, exist_ok = True)

    # 2. Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should and with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # 3. Save the model with state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(
        obj = model.state_dict(),
        f = model_save_path
    )

Writing going_modular/utils.py


# Combine Things

Combine all the python scripts created above into a ```train.py``` file so that we can train a PyTorch model using a single line of command.

In [24]:
%%writefile going_modular/train.py
'''
Trains a PyTorch image classification model using device-agnostic code
'''
import os
import torch
import data_setup, engine, model_builder, utils
from torch import nn
from torchvision import transforms

# Setup hyperparams
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories (For getting data)
train_dir = "data/pizza-steak-sushi/train"
test_dir = "data/pizza-steak-sushi/test"

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create DataLoaders (data_setup.py)
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE
)

# Create Model (model_builder.py)
model = model_builder.TinyVGG(
    input_shape = 3,
    hidden_units = HIDDEN_UNITS,
    output_shape = len(class_names)
).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = LEARNING_RATE
)

# Training (engine.py)
engine.train(
    model = model,
    train_dataloader = train_dataloader,
    test_dataloader = test_dataloader,
    loss_fn = loss_fn,
    optimizer = optimizer,
    epochs = NUM_EPOCHS,
    device = device
)

# Save model (utils.py)
utils.save_model(
    model = model,
    target_dir = "models",
    model_name = "05_going_modular_script_mode_tinyvgg_model.pth"
)

Overwriting going_modular/train.py


In [27]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1116 | train_acc: 0.2656 | test_loss: 1.1063 | test_acc: 0.2604
 20% 1/5 [00:03<00:12,  3.12s/it]Epoch: 2 | train_loss: 1.0946 | train_acc: 0.4258 | test_loss: 1.0943 | test_acc: 0.2604
 40% 2/5 [00:05<00:08,  2.67s/it]Epoch: 3 | train_loss: 1.0936 | train_acc: 0.4258 | test_loss: 1.0934 | test_acc: 0.2604
 60% 3/5 [00:07<00:04,  2.22s/it]Epoch: 4 | train_loss: 1.0965 | train_acc: 0.3047 | test_loss: 1.0888 | test_acc: 0.2604
 80% 4/5 [00:08<00:01,  2.00s/it]Epoch: 5 | train_loss: 1.0897 | train_acc: 0.3047 | test_loss: 1.0757 | test_acc: 0.2604
100% 5/5 [00:10<00:00,  2.10s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth
